# In-Sample Evaluation and Cross-Validation

## Overfitting
Another potential source of errors in models is __overfitting__.  __Overfitting__ is when the model is so excessively complex that it starts to catch random noise instead of describing the true underlying relationships.  This is typically manifested with a model that evaluates as more accurate than it really is.  In most situations there isn’t a perfect model, some error is to be expected.  __Overfitting__ is extremely common and easy to do, but there are ways to guard against it.  The main way is through how the model is evaluated.

So far, all the data has been used to train the model then the model is checked to see how it is doing.  In doing so, some of the danger of that approach is ill-advised.  An elaborate model will pick up on the nuances of the data that are just from random noise.  If the model is evaluated on training data, the ability to pick noise will be returned as accuracy.  In reality this isn’t the case and doesn’t depict how the model should be evaluated.  Generally, you don’t care about predicting things you already know.  You care about other data, new information, or other situations.  This is why testing with training data isn’t what you want to do.


## Holdout Groups
The simplest way to combat __overfitting__ is with a __holdout group__ (aka “holdback group”).   This means that you do not include all your data in the training set, instead reserving some of it exclusively for testing.  While there is a cost to having less training data, the evaluation will be far more reliable.

When directly comparing two models that are base on different techniques or different specifications, the holdout method combats __overfitting__.  Overfit models will see a drop-in success rate outside of their training data, and so their performance will not be artificially inflated as it would be if the model was trained and evaluated using the whole data set.  This is because the model got really good at matching the patterns within the data it was trained with but didn’t actually learn the things that matter but random noise.  When the model tries to match that random noise on new data its accuracy suffers.

The amount of data chosen to keep in a holdout depends on how much and what kind of data you begin with as well as what kind of model you are training.  Check and see how much variance the model has as more data is added as well as how much data it would take to maintain a reasonably representative test sample.  It is, however, a balance, 30% is common, but can range from 50% to 1% of the original dataset.

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import sklearn

from sklearn.model_selection import cross_val_score # New in this module
from sklearn.model_selection import KFold
from scipy import stats

plt.style.use("default")

In [74]:
# Grab and process the raw data.
data_path = (
    "https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/"
             "master/sms_spam_collection/SMSSpamCollection"
)

sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
sms_raw.columns = ['spam', 'message']

# Enumerate our spammy keywords.
keywords = ['click', 'offer', 'winner', 'buy', 'free', 'cash', 'urgent']

for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

sms_raw['allcaps'] = sms_raw.message.str.isupper()
sms_raw['spam'] = (sms_raw['spam'] == 'spam')
data = sms_raw[keywords + ['allcaps']]
target = sms_raw['spam']

from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

In [36]:
# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

With 20% Holdout: 0.884304932735426
Testing on Sample: 0.8916008614501076


The scores look consistent and it doesn't seem like the model is __overfitting__.  This is partly because the model is so simple.  This may be but the model should be evaluated to see if any other issues are lurking.

## Cross-Validation

Cross-Validation is a more robust version of holdout groups.  Instead of creating a single holdout, several are created.  

It works like this: start by breaking up the data into several equally sized pieces, or __folds__, creating _x_ folds.  You then go through the training and testing process _x_ times, each time with a different fold held out from the training data and used as the test set.  The number of folds created is subjective, but it depends ho how much data is desired in the testing set.  At its most extreme, the same number of folds are created as there are observations in the dataset.  This kind of cross-validation has a special name: __Leave One Out__.  Leave One Out is useful if there is a concern about single observations skewing the model vs. large folds that combat more general overfitting.

In [5]:
cross_val_score(bnb, data, target, cv=10)

array([0.89784946, 0.89426523, 0.89426523, 0.890681  , 0.89605735,
       0.89048474, 0.88150808, 0.89028777, 0.88489209, 0.89568345])

The array that `cross_val_score` returns is a series of accuracy scores with a different hold out group each time.  If the model is __overfitting__ as a variable amount, the scores will fluctuate.  For the current model, the scores stayed consistent and are what you wanted to see.  Further documentation on SKLearn’s cross-validation can be found here: http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators-with-stratification-based-on-class-labels.  

The outputs from the function are somewhat limited.  By default, `cross_val_score` uses the `score` method.  You can adjust what is returned, but you don’t get all the error types or outputs you may be interested in.  That’s why it is not uncommon for people to code up their own cross-validation.

In [50]:
# Implement your own cross validation with your spam model.
scores = cross_val_score(bnb, data, target, cv=28)
print(f"Accuracy: {scores.mean():.2f}% (+/-{scores.std()*2:.2f}) Mode: {stats.mode(scores)[0][0]:.2f}%")

Accuracy: 0.89% (+/-0.03) Mode: 0.88%


## What’s a good score?
When considering the current model, it has been returning accuracy scores around 0.89.  Intuitively, that seems like a pretty good score, but there could be type errors or a class imbalance.  For this model, both are at play here.  Deploying the Confusion Matrix you created earlier, apply it to the model and see what kinds of errors are in action.


In [116]:
# Perform your additional evaluation here.
# So that I can wrap my head around this X=data & y=target
X = data
y = target

# Set the number of folds (training and test samples) 
kf = KFold(n_splits=5)

# Counter for pinting
i=0

# Train and test using the KFolds
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    i+=1
    # Create the train samples from the fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]

    # Create the test samples from the fold
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Locate the appropriate rows in the target series
    target_ref = target.iloc[train_index]

    # Train the model using the train and test data
    y_pred = bnb.fit(X_train, y_train).predict(X_train)

    # Create a Confusion Matrix to report on model training
    confusion_matrix = dict(
        false_positives=((~target_ref)&(y_pred)).sum(),
        false_negatives=((target_ref)&(~y_pred)).sum(),
        y_pred_correct_pos=((target_ref)&(y_pred)).sum(),
        y_pred_correct_neg=((~target_ref)&(~y_pred)).sum(),
        target_pos=(target_ref).sum(),
        target_neg=(~target_ref).sum(),
        sensitivity=(((target_ref)&(y_pred)).sum()/(target_ref).sum())*100,     
        specificity=(((~target_ref)&(~y_pred)).sum()/(~target_ref).sum())*100, 
        total_errors=(target_ref != y_pred).sum()
    )

    # Generate a report of model training
    print(f"""Fold {i}:
Total Errors: {confusion_matrix["total_errors"]}, False Negatives: {confusion_matrix["false_negatives"]}, False Positives: {confusion_matrix["false_positives"]}, Sensitivity: {confusion_matrix["sensitivity"]:,.4f}% Specificity: {confusion_matrix["specificity"]:,.4f}%
Testing on Sample: {bnb.fit(X_test, y_test).score(X_test, y_test)}
""")

Fold 1:
Total Errors: 473, False Negatives: 432, False Positives: 41, Sensitivity: 25.3886% Specificity: 98.9428%
Testing on Sample: 0.8825112107623319

Fold 2:
Total Errors: 493, False Negatives: 446, False Positives: 47, Sensitivity: 26.4026% Specificity: 98.7795%
Testing on Sample: 0.9013452914798207

Fold 3:
Total Errors: 492, False Negatives: 446, False Positives: 46, Sensitivity: 26.8852% Specificity: 98.8046%
Testing on Sample: 0.900359066427289

Fold 4:
Total Errors: 475, False Negatives: 431, False Positives: 44, Sensitivity: 27.0728% Specificity: 98.8622%
Testing on Sample: 0.8842010771992819

Fold 5:
Total Errors: 483, False Negatives: 443, False Positives: 40, Sensitivity: 26.4120% Specificity: 98.9627%
Testing on Sample: 0.8922800718132855



## Thinking like a Data Scientist

The validation method chosen depends upon the kind of data being considered and any concerns about the model’s performance.  Remember the model is trained to fit the data it has been fed, so if the situation changes the model will become less accurate.  For example, if there are seasonal changes to an observed variable but the model was trained with only one month’s data, there will be a problem.  This can be tested by seeing how accurate the model is with a specific time period as a holdout, rather than a random sample.


## Overfitting and Naïve Bayes

Overfitting is always possible, but some models are more susceptible to it than others.  Naïve Bayes is actually pretty good for avoiding overfitting.  This is largely because the assumptions are so simple, particularly the assumed independence between any two independent variables.  One of the sources of overfitting is when a model tries to map complex interactions between variables that aren’t really there or significant.  Naïve Bayes cannot do this because it assumes, they are all independent and therefore not interacting.  It’s a nice characteristic at times, but it does mean it doesn’t take into account how the model’s features affect each other.


## Outside Resources

- Train/Test Split and Cross Validation in Python: https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

- The Confusion Matrix: Unveiled: https://towardsdatascience.com/the-confusion-matrix-unveiled-2d030136be40

- The ROC Curve: Unveiled: https://towardsdatascience.com/the-roc-curve-unveiled-81296e1577b